<a href="https://colab.research.google.com/github/aaronpetryio/random/blob/main/mmlm_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os
import re

In [2]:
!pip install -q kaggle

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"aaronpetryio","key":"ac3f073bb1dfa9030d82b13d6f357d21"}'}

In [4]:
!mkdir ~/.kaggle

In [5]:
!cp kaggle.json ~/.kaggle/

In [6]:
!chmod 600 ~/.kaggle/kaggle.json

In [85]:
!kaggle datasets list -s ncaa-women

ref                                                      title                                        size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------  ------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
raddar/ncaa-women-538-team-ratings                       NCAA Women 538 team ratings                   4KB  2022-03-17 08:15:51            429         84  0.47058824       
tylerwiddison/ncaa-womens-volleyball-boxscores-20122019  NCAA Womens Volleyball boxscores 2012-2019   22MB  2020-05-21 23:19:54            183          6  0.88235295       
whenamancodes/ncaa-women-538-team-ratings                NCAA Women 538 Team Ratings                   4KB  2022-10-27 07:25:52              5          4  0.9411765        
raddar/wncaa-basketball-espn-ratings                     WNCAA basketball ESPN ratings                 2KB  2022-10-23 11:42:22        

In [86]:
!kaggle datasets download -d raddar/ncaa-women-538-team-ratings

  0% 0.00/3.55k [00:00<?, ?B/s]
100% 3.55k/3.55k [00:00<00:00, 2.86MB/s]


In [87]:
!unzip ncaa-women-538-team-ratings.zip

Archive:  ncaa-women-538-team-ratings.zip
  inflating: 538ratingsWomen.csv     


In [7]:
!kaggle competitions download -c warmup-round-march-machine-learning-mania-2023

100% 29.2M/29.2M [00:00<00:00, 153MB/s]
100% 29.2M/29.2M [00:00<00:00, 150MB/s]


In [8]:
!unzip warmup-round-march-machine-learning-mania-2023.zip

Archive:  warmup-round-march-machine-learning-mania-2023.zip
  inflating: Cities.csv              
  inflating: Conferences.csv         
  inflating: MConferenceTourneyGames.csv  
  inflating: MGameCities.csv         
  inflating: MMasseyOrdinals.csv     
  inflating: MNCAATourneyCompactResults.csv  
  inflating: MNCAATourneyDetailedResults.csv  
  inflating: MNCAATourneySeedRoundSlots.csv  
  inflating: MNCAATourneySeeds.csv   
  inflating: MNCAATourneySlots.csv   
  inflating: MRegularSeasonCompactResults.csv  
  inflating: MRegularSeasonDetailedResults.csv  
  inflating: MSeasons.csv            
  inflating: MSecondaryTourneyCompactResults.csv  
  inflating: MSecondaryTourneyTeams.csv  
  inflating: MTeamCoaches.csv        
  inflating: MTeamConferences.csv    
  inflating: MTeamSpellings.csv      
  inflating: MTeams.csv              
  inflating: SampleSubmissionWarmup.csv  
  inflating: WGameCities.csv         
  inflating: WNCAATourneyCompactResults.csv  
  inflating: WNCAATourn

In [9]:
df_seeds = pd.read_csv("WNCAATourneySeeds.csv")
df_seeds.head()

,Season,Seed,TeamID
0,1998,W01,3330
1,1998,W02,3163
2,1998,W03,3112
3,1998,W04,3301
4,1998,W05,3272


In [10]:
df_season_results = pd.read_csv("WRegularSeasonCompactResults.csv")
df_season_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1998,18,3104,91,3202,41,H,0
1,1998,18,3163,87,3221,76,H,0
2,1998,18,3222,66,3261,59,H,0
3,1998,18,3307,69,3365,62,H,0
4,1998,18,3349,115,3411,35,H,0


In [11]:
df_season_results['ScoreGap'] = df_season_results['WScore'] - df_season_results['LScore']

In [12]:
df_season_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,ScoreGap
0,1998,18,3104,91,3202,41,H,0,50
1,1998,18,3163,87,3221,76,H,0,11
2,1998,18,3222,66,3261,59,H,0,7
3,1998,18,3307,69,3365,62,H,0,7
4,1998,18,3349,115,3411,35,H,0,80


In [13]:
num_win = df_season_results.groupby(['Season', 'WTeamID']).count()
num_win = num_win.reset_index()[['Season', 'WTeamID', 'DayNum']].rename(columns={"DayNum": "NumWins", "WTeamID": "TeamID"})

In [14]:
num_win.head()

,Season,TeamID,NumWins
0,1998,3102,4
1,1998,3103,11
2,1998,3104,21
3,1998,3106,6
4,1998,3108,12


In [15]:
num_loss = df_season_results.groupby(['Season', 'LTeamID']).count()
num_loss = num_loss.reset_index()[['Season', 'LTeamID', 'DayNum']].rename(columns={"DayNum": "NumLosses", "LTeamID":"TeamID"})

In [16]:
num_loss.head()

,Season,TeamID,NumLosses
0,1998,3102,20
1,1998,3103,18
2,1998,3104,9
3,1998,3106,15
4,1998,3108,11


In [17]:
gap_win = df_season_results.groupby(['Season', 'WTeamID']).mean().reset_index()
gap_win = gap_win[['Season', 'WTeamID', 'ScoreGap']].rename(columns={"ScoreGap": "GapWins", "WTeamID": "TeamID"})

In [18]:
gap_win.head()

,Season,TeamID,GapWins
0,1998,3102,15.500000
1,1998,3103,15.000000
2,1998,3104,21.619048
3,1998,3106,8.333333
4,1998,3108,17.750000


In [19]:
gap_loss = df_season_results.groupby(['Season', 'LTeamID']).mean().reset_index()
gap_loss = gap_loss[['Season', 'LTeamID', 'ScoreGap']].rename(columns={"ScoreGap": "GapLosses", "LTeamID": "TeamID"})

In [30]:
df_feature_seasons_w = df_season_results.groupby(['Season', 'WTeamID']).count().reset_index()[['Season', 'WTeamID']].rename(columns={"WTeamID": "TeamID"})
df_feature_seasons_l = df_season_results.groupby(['Season', 'LTeamID']).count().reset_index()[['Season', 'LTeamID']].rename(columns={"LTeamID": "TeamID"})

In [62]:
df_features_season = pd.concat([df_feature_seasons_w, df_feature_seasons_l], 0).drop_duplicates().sort_values(['Season', 'TeamID']).reset_index(drop=True)

<ipython-input-62-7918e6a1aef6>:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  df_features_season = pd.concat([df_feature_seasons_w, df_feature_seasons_l], 0).drop_duplicates().sort_values(['Season', 'TeamID']).reset_index(drop=True)


In [64]:
df_features_season.head()

,Season,TeamID
0,1998,3102
1,1998,3103
2,1998,3104
3,1998,3106
4,1998,3108


In [65]:
df_features_season = df_features_season.merge(num_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(num_loss, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(gap_win, on=['Season', 'TeamID'], how='left')
df_features_season = df_features_season.merge(gap_loss, on=['Season', 'TeamID'], how='left')

In [66]:
df_features_season.head()

,Season,TeamID,NumWins,NumLosses,GapWins,GapLosses
0,1998,3102,4.0,20.0,15.500000,27.850000
1,1998,3103,11.0,18.0,15.000000,18.611111
2,1998,3104,21.0,9.0,21.619048,5.666667
3,1998,3106,6.0,15.0,8.333333,14.466667
4,1998,3108,12.0,11.0,17.750000,16.636364


In [67]:
df_features_season.fillna(0, inplace=True)

In [68]:
df_features_season.head()

,Season,TeamID,NumWins,NumLosses,GapWins,GapLosses
0,1998,3102,4.0,20.0,15.500000,27.850000
1,1998,3103,11.0,18.0,15.000000,18.611111
2,1998,3104,21.0,9.0,21.619048,5.666667
3,1998,3106,6.0,15.0,8.333333,14.466667
4,1998,3108,12.0,11.0,17.750000,16.636364


In [69]:
df_features_season['WinRatio'] = df_features_season['NumWins'] / (df_features_season['NumWins'] + df_features_season['NumLosses'])

In [70]:
df_features_season.head()

,Season,TeamID,NumWins,NumLosses,GapWins,GapLosses,WinRatio
0,1998,3102,4.0,20.0,15.500000,27.850000,0.166667
1,1998,3103,11.0,18.0,15.000000,18.611111,0.379310
2,1998,3104,21.0,9.0,21.619048,5.666667,0.700000
3,1998,3106,6.0,15.0,8.333333,14.466667,0.285714
4,1998,3108,12.0,11.0,17.750000,16.636364,0.521739


In [71]:
df_features_season['GapAvg'] = (
  (df_features_season['NumWins'] * df_features_season['GapWins'] - df_features_season['NumLosses'] * df_features_season['GapLosses'])
  / (df_features_season['NumWins'] + df_features_season['NumLosses'])  
)

In [77]:
df_features_season.head()

,Season,TeamID,WinRatio,GapAvg
0,1998,3102,0.166667,-20.625000
1,1998,3103,0.379310,-5.862069
2,1998,3104,0.700000,13.433333
3,1998,3106,0.285714,-7.952381
4,1998,3108,0.521739,1.304348


In [76]:
df_features_season.drop(['NumWins', 'NumLosses', 'GapWins', 'GapLosses'], axis=1, inplace=True)

KeyError: ignored

In [79]:
df_features_season.head()

,Season,TeamID,WinRatio,GapAvg
0,1998,3102,0.166667,-20.625000
1,1998,3103,0.379310,-5.862069
2,1998,3104,0.700000,13.433333
3,1998,3106,0.285714,-7.952381
4,1998,3108,0.521739,1.304348


In [80]:
df_tourney_results = pd.read_csv("WNCAATourneyCompactResults.csv")

In [81]:
df_tourney_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1998,137,3104,94,3422,46,H,0
1,1998,137,3112,75,3365,63,H,0
2,1998,137,3163,93,3193,52,H,0
3,1998,137,3198,59,3266,45,H,0
4,1998,137,3203,74,3208,72,A,0


In [82]:
df_tourney_results.drop(['NumOT', 'WLoc'], axis=1, inplace=True)

In [83]:
df_tourney_results.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore
0,1998,137,3104,94,3422,46
1,1998,137,3112,75,3365,63
2,1998,137,3163,93,3193,52
3,1998,137,3198,59,3266,45
4,1998,137,3203,74,3208,72


In [89]:
df_538 = pd.read_csv("538ratingsWomen.csv")
df_538.drop('TeamName', axis=1, inplace=True)

In [91]:
df_538.head()

,Season,TeamID,538rating
0,2016,3163,113.09
1,2016,3124,101.59
2,2016,3323,101.07
3,2016,3376,101.07
4,2016,3268,99.01


In [168]:
df = df_tourney_results.copy()

In [169]:
df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore
0,1998,137,3104,94,3422,46
1,1998,137,3112,75,3365,63
2,1998,137,3163,93,3193,52
3,1998,137,3198,59,3266,45
4,1998,137,3203,74,3208,72


In [170]:
df = df[df['Season'] >= 2016].reset_index(drop=True)

In [171]:
df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore
0,2016,137,3107,61,3196,59
1,2016,137,3113,74,3308,52
2,2016,137,3120,68,3385,57
3,2016,137,3124,89,3225,59
4,2016,137,3177,97,3241,67


In [172]:
df_seeds.head()

,Season,Seed,TeamID
0,1998,W01,3330
1,1998,W02,3163
2,1998,W03,3112
3,1998,W04,3301
4,1998,W05,3272


In [173]:
df = pd.merge(
    df,
    df_seeds,
    how='left',
    left_on=['Season', 'WTeamID'],
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'Seed':'SeedW'})

In [174]:
df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,SeedW
0,2016,137,3107,61,3196,59,Z12
1,2016,137,3113,74,3308,52,Z02
2,2016,137,3120,68,3385,57,X09
3,2016,137,3124,89,3225,59,X01
4,2016,137,3177,97,3241,67,X06


In [175]:
df = pd.merge(
    df,
    df_seeds,
    how='left', 
    left_on=['Season', 'LTeamID'],
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'Seed':'SeedL'})

In [176]:
df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,SeedW,SeedL
0,2016,137,3107,61,3196,59,Z12,Z05
1,2016,137,3113,74,3308,52,Z02,Z15
2,2016,137,3120,68,3385,57,X09,X08
3,2016,137,3124,89,3225,59,X01,X16
4,2016,137,3177,97,3241,67,X06,X11


In [177]:
def treat_seed(seed):
  return int(re.sub("[^0-9]", "", seed))

In [178]:
df['SeedW'] = df['SeedW'].apply(treat_seed)

In [179]:
df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,SeedW,SeedL
0,2016,137,3107,61,3196,59,12,Z05
1,2016,137,3113,74,3308,52,2,Z15
2,2016,137,3120,68,3385,57,9,X08
3,2016,137,3124,89,3225,59,1,X16
4,2016,137,3177,97,3241,67,6,X11


In [180]:
df['SeedL'] = df['SeedL'].apply(treat_seed)

In [181]:
df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,SeedW,SeedL
0,2016,137,3107,61,3196,59,12,5
1,2016,137,3113,74,3308,52,2,15
2,2016,137,3120,68,3385,57,9,8
3,2016,137,3124,89,3225,59,1,16
4,2016,137,3177,97,3241,67,6,11


In [182]:
df_features_season.head()

,Season,TeamID,WinRatio,GapAvg
0,1998,3102,0.166667,-20.625000
1,1998,3103,0.379310,-5.862069
2,1998,3104,0.700000,13.433333
3,1998,3106,0.285714,-7.952381
4,1998,3108,0.521739,1.304348


In [183]:
df = pd.merge(
    df,
    df_features_season,
    how='left',
    left_on=['Season', 'WTeamID'],
    right_on=['Season', 'TeamID']
).rename(columns={
    'WinRatio': 'WinRatioW',
    'GapAvg': 'GapAvgW'
}).drop(columns='TeamID', axis=1)

In [184]:
df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,SeedW,SeedL,WinRatioW,GapAvgW
0,2016,137,3107,61,3196,59,12,5,0.870968,18.677419
1,2016,137,3113,74,3308,52,2,15,0.806452,9.258065
2,2016,137,3120,68,3385,57,9,8,0.612903,3.032258
3,2016,137,3124,89,3225,59,1,16,0.970588,24.617647
4,2016,137,3177,97,3241,67,6,11,0.757576,16.272727


In [185]:
df = pd.merge(
    df,
    df_features_season,
    how='left',
    left_on=['Season', 'LTeamID'],
    right_on=['Season', 'TeamID']
).rename(columns={
    'WinRatio': 'WinRatioL',
    'GapAvg': 'GapAvgL'
}).drop(columns='TeamID', axis=1)

In [186]:
df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,SeedW,SeedL,WinRatioW,GapAvgW,WinRatioL,GapAvgL
0,2016,137,3107,61,3196,59,12,5,0.870968,18.677419,0.733333,8.700000
1,2016,137,3113,74,3308,52,2,15,0.806452,9.258065,0.857143,8.964286
2,2016,137,3120,68,3385,57,9,8,0.612903,3.032258,0.718750,6.812500
3,2016,137,3124,89,3225,59,1,16,0.970588,24.617647,0.709677,10.612903
4,2016,137,3177,97,3241,67,6,11,0.757576,16.272727,0.843750,15.406250


In [187]:
df_538.head()

,Season,TeamID,538rating
0,2016,3163,113.09
1,2016,3124,101.59
2,2016,3323,101.07
3,2016,3376,101.07
4,2016,3268,99.01


In [188]:
df = pd.merge(
    df,
    df_538,
    how='left',
    left_on=['Season', 'WTeamID'],
    right_on=['Season', 'TeamID']
).rename(columns={'538rating': '538ratingW'}).drop(columns='TeamID', axis=1)

In [189]:
df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,SeedW,SeedL,WinRatioW,GapAvgW,WinRatioL,GapAvgL,538ratingW
0,2016,137,3107,61,3196,59,12,5,0.870968,18.677419,0.733333,8.700000,80.90
1,2016,137,3113,74,3308,52,2,15,0.806452,9.258065,0.857143,8.964286,91.32
2,2016,137,3120,68,3385,57,9,8,0.612903,3.032258,0.718750,6.812500,80.17
3,2016,137,3124,89,3225,59,1,16,0.970588,24.617647,0.709677,10.612903,101.59
4,2016,137,3177,97,3241,67,6,11,0.757576,16.272727,0.843750,15.406250,90.08


In [190]:
df = pd.merge(
    df,
    df_538,
    how='left',
    left_on=['Season', 'LTeamID'],
    right_on=['Season', 'TeamID']
).rename(columns={'538rating': '538ratingL'}).drop(columns='TeamID', axis=1)

In [191]:
df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,SeedW,SeedL,WinRatioW,GapAvgW,WinRatioL,GapAvgL,538ratingW,538ratingL
0,2016,137,3107,61,3196,59,12,5,0.870968,18.677419,0.733333,8.700000,80.90,83.36
1,2016,137,3113,74,3308,52,2,15,0.806452,9.258065,0.857143,8.964286,91.32,71.42
2,2016,137,3120,68,3385,57,9,8,0.612903,3.032258,0.718750,6.812500,80.17,82.19
3,2016,137,3124,89,3225,59,1,16,0.970588,24.617647,0.709677,10.612903,101.59,72.69
4,2016,137,3177,97,3241,67,6,11,0.757576,16.272727,0.843750,15.406250,90.08,83.04


In [192]:
df_copy = df.copy()

In [193]:
def add_losing_matches(df):
    win_rename = {
        "WTeamID": "TeamIdA", 
        "WScore" : "ScoreA", 
        "LTeamID" : "TeamIdB",
        "LScore": "ScoreB",
     }
    win_rename.update({c : c[:-1] + "A" for c in df.columns if c.endswith('W')})
    win_rename.update({c : c[:-1] + "B" for c in df.columns if c.endswith('L')})
    
    lose_rename = {
        "WTeamID": "TeamIdB", 
        "WScore" : "ScoreB", 
        "LTeamID" : "TeamIdA",
        "LScore": "ScoreA",
    }
    lose_rename.update({c : c[:-1] + "B" for c in df.columns if c.endswith('W')})
    lose_rename.update({c : c[:-1] + "A" for c in df.columns if c.endswith('L')})
    
    win_df = df.copy()
    lose_df = df.copy()
    
    win_df = win_df.rename(columns=win_rename)
    lose_df = lose_df.rename(columns=lose_rename)
    
    return pd.concat([win_df, lose_df], 0, sort=False)

In [194]:
df  = add_losing_matches(df)

<ipython-input-193-4366e12a09bf>:26: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  return pd.concat([win_df, lose_df], 0, sort=False)


In [195]:
df.head()

,Season,DayNum,TeamIdA,ScoreA,TeamIdB,ScoreB,SeedA,SeedB,WinRatioA,GapAvgA,WinRatioB,GapAvgB,538ratingA,538ratingB
0,2016,137,3107,61,3196,59,12,5,0.870968,18.677419,0.733333,8.700000,80.90,83.36
1,2016,137,3113,74,3308,52,2,15,0.806452,9.258065,0.857143,8.964286,91.32,71.42
2,2016,137,3120,68,3385,57,9,8,0.612903,3.032258,0.718750,6.812500,80.17,82.19
3,2016,137,3124,89,3225,59,1,16,0.970588,24.617647,0.709677,10.612903,101.59,72.69
4,2016,137,3177,97,3241,67,6,11,0.757576,16.272727,0.843750,15.406250,90.08,83.04


In [196]:
cols_to_diff = [
    'Seed', 'WinRatio', 'GapAvg', '538Rating'
]

In [197]:
cols_to_diff = [
    'Seed', 'WinRatio', 'GapAvg', '538rating'
]

for col in cols_to_diff:
    df[col + 'Diff'] = df[col + 'A'] - df[col + 'B']

In [199]:
df.head()

,Season,DayNum,TeamIdA,ScoreA,TeamIdB,ScoreB,SeedA,SeedB,WinRatioA,GapAvgA,WinRatioB,GapAvgB,538ratingA,538ratingB,SeedDiff,WinRatioDiff,GapAvgDiff,538ratingDiff
0,2016,137,3107,61,3196,59,12,5,0.870968,18.677419,0.733333,8.700000,80.90,83.36,7,0.137634,9.977419,-2.46
1,2016,137,3113,74,3308,52,2,15,0.806452,9.258065,0.857143,8.964286,91.32,71.42,-13,-0.050691,0.293779,19.90
2,2016,137,3120,68,3385,57,9,8,0.612903,3.032258,0.718750,6.812500,80.17,82.19,1,-0.105847,-3.780242,-2.02
3,2016,137,3124,89,3225,59,1,16,0.970588,24.617647,0.709677,10.612903,101.59,72.69,-15,0.260911,14.004744,28.90
4,2016,137,3177,97,3241,67,6,11,0.757576,16.272727,0.843750,15.406250,90.08,83.04,-5,-0.086174,0.866477,7.04


Test Data

In [214]:
df_test = pd.read_csv("SampleSubmissionWarmup.csv")

In [215]:
df_test.head()

,ID,Pred
0,2017_1101_1102,0.5
1,2017_1101_1103,0.5
2,2017_1101_1104,0.5
3,2017_1101_1105,0.5
4,2017_1101_1106,0.5


In [216]:
df_test['Season'] = df_test['ID'].apply(lambda x: int(x.split('_')[0]))
df_test['TeamIdA'] = df_test['ID'].apply(lambda x: int(x.split('_')[1]))
df_test['TeamIdB'] = df_test['ID'].apply(lambda x: int(x.split('_')[2]))

In [209]:
df_test.head()

,ID,Pred,Season,TeamIdA,TeamIdB
0,2017_1101_1102,0.5,2017,1101,1102
1,2017_1101_1103,0.5,2017,1101,1103
2,2017_1101_1104,0.5,2017,1101,1104
3,2017_1101_1105,0.5,2017,1101,1105
4,2017_1101_1106,0.5,2017,1101,1106


In [210]:
df_test = pd.merge(
    df_test,
    df_seeds,
    how='left',
    left_on=['Season', 'TeamIdA'],
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'Seed': 'SeedA'})

In [211]:
df_test = pd.merge(
    df_test,
    df_seeds,
    how='left',
    left_on=['Season', 'TeamIdB'],
    right_on=['Season', 'TeamID']
).drop('TeamID', axis=1).rename(columns={'Seed': 'SeedB'})

In [213]:
df_test.head()

,ID,Pred,Season,TeamIdA,TeamIdB,SeedA,SeedB
0,2017_1101_1102,0.5,2017,1101,1102,NaN,NaN
1,2017_1101_1103,0.5,2017,1101,1103,NaN,NaN
2,2017_1101_1104,0.5,2017,1101,1104,NaN,NaN
3,2017_1101_1105,0.5,2017,1101,1105,NaN,NaN
4,2017_1101_1106,0.5,2017,1101,1106,NaN,NaN
